# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv

import edurel.utils.dbcon as dbcu
import edurel.utils.duckdb as dbu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


# Database

In [3]:
db = dbcu.company_en()

In [4]:
sql = "select * from project"
db.print(sql)

┌───────┬────────────────────────┬───────────────┐
│  PID  │         Title          │    Budget     │
│ int32 │        varchar         │ decimal(13,2) │
├───────┼────────────────────────┼───────────────┤
│     1 │ Strategy               │      80000.00 │
│     2 │ CRM                    │      20000.00 │
│     3 │ Plant Restructurierung │      50000.00 │
│     4 │ Sales                  │          NULL │
└───────┴────────────────────────┴───────────────┘



In [5]:
db.tablenames()

['EmpProj', 'Employee', 'OrgUnit', 'Project']

In [6]:
db.columns("Project")

[{'columnname': 'PID', 'type': 'INTEGER', 'nullable': False},
 {'columnname': 'Title', 'type': 'VARCHAR', 'nullable': False},
 {'columnname': 'Budget', 'type': 'DECIMAL(13,2)', 'nullable': True}]

In [7]:
db.primary_keys("Project")

['PID']

In [8]:
print(db.yaml())

tables:
- tablename: EmpProj
  columns:
  - columnname: EID
    type: INTEGER
    nullable: false
  - columnname: PID
    type: INTEGER
    nullable: false
  - columnname: NoOfHoursPerWeek
    type: INTEGER
    nullable: false
  primary_key:
  - EID
  - PID
  foreign_keys:
  - fkname: fk_EmpProj_Employee_EID_1
    sourcecolumns:
    - EID
    targettable: Employee
    targetcolumns:
    - EID
  - fkname: fk_EmpProj_Project_PID_2
    sourcecolumns:
    - PID
    targettable: Project
    targetcolumns:
    - PID
- tablename: Employee
  columns:
  - columnname: EID
    type: INTEGER
    nullable: false
  - columnname: OUID
    type: INTEGER
    nullable: false
  - columnname: LastName
    type: VARCHAR
    nullable: false
  - columnname: Hiredate
    type: DATE
    nullable: false
  - columnname: Salary
    type: DECIMAL(9,2)
    nullable: false
  - columnname: Bonus
    type: DECIMAL(9,2)
    nullable: true
  primary_key:
  - EID
  foreign_keys:
  - fkname: fk_Employee_OrgUnit_OUID_1
   